In [1]:
import pandas as pd

In [2]:
file = '/Users/Lisa/_MLB_Prediction_capstone/cleaned_batting_df.csv'
file2 = '/Users/Lisa/_MLB_Prediction_capstone/cleaned_piching_df.csv'
file3 = '/Users/Lisa/_MLB_Prediction_capstone/cleaned_game_df.csv'

In [3]:
batting_df = pd.read_csv(file)
pitching_df = pd.read_csv(file2)
game_df = pd.read_csv(file3)

In [4]:
batting_df.drop('Unnamed: 0', axis=1, inplace=True)
pitching_df.drop('Unnamed: 0', axis=1, inplace=True)
game_df.drop('Unnamed: 0', axis=1, inplace=True)

## Batting Feature engineering ideas
- All of these stats will be on a PER PLATE APPEARANCE then divide by 162 to get PER GAME STATS
    - Team stats will be summed up and averaged - Will ultimately be compared to a teams 7 day trends history
        - 'totalBases' / total 'plateAppearances' - DONE
        - 'leftonbase'/ total 'plateAppearances' - DONE
        - 'runs' / total 'plateAppearances' - DONE
        - 'strikeOuts' / total 'atBats' - DONE
        - 'baseOnBalls' + 'intentionalWalks' / total 'plateAppearances' - DONE
            - 'hits' / total 'atBats' --- Leverage for game_df. get team stats per game...
            - 'obp' / # of batters on team --- Same as above
        - 'rbi' / total 'atBats' - DONE

In [5]:
batting_df.head()

,year,playerName,teamAbbrev,plateAppearances,totalBases,leftOnBase,runs,strikeOuts,baseOnBalls,intentionalWalks,hits,avg,atBats,obp,slg,ops,groundIntoDoublePlay,rbi
0,2009,Aaron Hill,TOR,734,340,286,103,98,42,1,195,0.286,682,0.330,0.499,0.829,17,108
1,2009,Jimmy Rollins,PHI,725,284,224,100,70,44,1,168,0.250,672,0.296,0.423,0.719,7,77
2,2009,Orlando Cabrera,MIN,708,255,244,83,71,36,1,186,0.284,656,0.316,0.389,0.705,22,77
3,2009,Nick Markakis,BAL,711,291,286,94,98,56,0,188,0.293,642,0.347,0.453,0.801,12,101
4,2009,Ichiro Suzuki,SEA,678,297,176,88,71,32,15,225,0.352,639,0.386,0.465,0.851,1,46


In [6]:
# MIA & FLO are different. Standardizing to MIA
batting_df.replace(to_replace='FLA', value='MIA', inplace=True)

In [7]:
# 31 teams showing up since it seems like there are some NaN values for team names. Will deal with should it become an issue...
len(batting_df.teamAbbrev.unique())

31

### Create New Features for TEAM stats

In [8]:
# THIS IS THE FRAMEWORK TO GENERATE TEAM STATISTICS...
# Creating a dataframe containing team annual Batting Averages
a = batting_df.groupby(['year', 'teamAbbrev'])['hits'].sum()
b = batting_df.groupby(['year', 'teamAbbrev'])['plateAppearances'].sum()
a = a.to_frame()
b = b.to_frame()
BA = a.join(b, on=['year', 'teamAbbrev'])
BA['team_BA'] = round((BA['hits']/BA['plateAppearances']), 3)
# BA = BA.reset_index()

In [9]:
# Total bases per plate appearance
# Commented out. Taking a different approach
# batting_df['Total_bases_per_plateappearance'] = (batting_df['totalBases'] / batting_df['plateAppearances'])

a = batting_df.groupby(['year', 'teamAbbrev'])['totalBases'].sum()
b = batting_df.groupby(['year', 'teamAbbrev'])['plateAppearances'].sum()
a = a.to_frame()
b = b.to_frame()
TB = a.join(b, on=['year', 'teamAbbrev'])
TB['total_bases_avg'] = round((TB['totalBases']/TB['plateAppearances']), 3)
# TB = TB.reset_index()

In [10]:
# LOB per plate appearnace
# Commented out. Taking a different approach
# batting_df['LOB_per_plateappearance'] = (batting_df['leftOnBase'] / batting_df['plateAppearances'])

a = batting_df.groupby(['year', 'teamAbbrev'])['leftOnBase'].sum()
b = batting_df.groupby(['year', 'teamAbbrev'])['plateAppearances'].sum()
a = a.to_frame()
b = b.to_frame()
LOB = a.join(b, on=['year', 'teamAbbrev'])
LOB['LOB_avg'] = round((LOB['leftOnBase']/LOB['plateAppearances']), 3)
# LOB = LOB.reset_index()

In [11]:
# Runs per plate appearance
# Commented out. Taking a different approach
#batting_df['runs_per_plateappearance'] = (batting_df['runs'] / batting_df['plateAppearances'])

a = batting_df.groupby(['year', 'teamAbbrev'])['runs'].sum()
b = batting_df.groupby(['year', 'teamAbbrev'])['plateAppearances'].sum()
a = a.to_frame()
b = b.to_frame()
RUNS = a.join(b, on=['year', 'teamAbbrev'])
RUNS['runs_avg'] = round((RUNS['runs']/RUNS['plateAppearances']), 3)
# RUNS = RUNS.reset_index()

In [12]:
# K's per AtBat
# Commented out. Taking a different approach
# batting_df['K_per_plateappearance'] = (batting_df['strikeOuts'] / batting_df['atBats'])

a = batting_df.groupby(['year', 'teamAbbrev'])['strikeOuts'].sum()
b = batting_df.groupby(['year', 'teamAbbrev'])['atBats'].sum()
a = a.to_frame()
b = b.to_frame()
K = a.join(b, on=['year', 'teamAbbrev'])
K['K_avg'] = round((K['strikeOuts']/K['atBats']), 3)
# K = K.reset_index()

In [13]:
# BB per plate appearance
# Commented out. Taking a different approach
# batting_df['BB+IBB_per_plateappearance'] = ((batting_df['baseOnBalls'] + 'took out IBB for simplicity' / batting_df['plateAppearances'])

a = batting_df.groupby(['year', 'teamAbbrev'])['baseOnBalls'].sum()
b = batting_df.groupby(['year', 'teamAbbrev'])['plateAppearances'].sum()
a = a.to_frame()
b = b.to_frame()
BB = a.join(b, on=['year', 'teamAbbrev'])
BB['BB_avg'] = round((BB['baseOnBalls']/BB['plateAppearances']), 3)
# BB = BB.reset_index()

In [14]:
# RBI per plate appearance
# Commented out. Taking a different approach
# batting_df['RBI_per_plateappearance'] = (batting_df['rbi'] / batting_df['atBats'])

a = batting_df.groupby(['year', 'teamAbbrev'])['rbi'].sum()
b = batting_df.groupby(['year', 'teamAbbrev'])['plateAppearances'].sum()
a = a.to_frame()
b = b.to_frame()
RBI = a.join(b, on=['year', 'teamAbbrev'])
RBI['RBI_avg'] = round((RBI['rbi']/RBI['plateAppearances']), 3)
# RBI = RBI.reset_index()

## Merge all the TEAM stats into one dataframe

In [15]:
team_stats_lst = [BA, TB, LOB, RUNS, K, BB, RBI]

In [16]:
team_stats = pd.concat(team_stats_lst, join='outer', axis=1)

In [17]:
team_stats = team_stats.reset_index()

In [18]:
team_stats

,year,teamAbbrev,hits,plateAppearances,team_BA,totalBases,plateAppearances,total_bases_avg,leftOnBase,plateAppearances,...,runs_avg,strikeOuts,atBats,K_avg,baseOnBalls,plateAppearances,BB_avg,rbi,plateAppearances,RBI_avg
0,2009,ARI,1310,5858,0.224,2195,5858,0.375,2225,5858,...,0.116,1224,5182,0.236,545,5858,0.093,664,5858,0.113
1,2009,ATL,1492,6510,0.229,2336,6510,0.359,2432,6510,...,0.119,1137,5678,0.200,637,6510,0.098,723,6510,0.111
2,2009,BAL,1367,5610,0.244,2107,5610,0.376,2132,5610,...,0.120,913,5073,0.180,449,5610,0.080,628,5610,0.112
3,2009,BOS,1915,8245,0.232,3151,8245,0.382,3223,8245,...,0.128,1435,7202,0.199,864,8245,0.105,1036,8245,0.126
4,2009,CHC,1392,6214,0.224,2228,6214,0.359,2458,6214,...,0.113,1176,5459,0.215,590,6214,0.095,679,6214,0.109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,2019,STL,1325,6105,0.217,2243,6105,0.367,2248,6105,...,0.124,1406,5393,0.261,555,6105,0.091,707,6105,0.116
326,2019,TB,1551,6835,0.227,2625,6835,0.384,2580,6835,...,0.122,1587,6103,0.260,601,6835,0.088,788,6835,0.115
327,2019,TEX,1298,5833,0.223,2260,5833,0.387,2107,5833,...,0.131,1492,5216,0.286,494,5833,0.085,714,5833,0.122
328,2019,TOR,1116,5403,0.207,2049,5403,0.379,1964,5403,...,0.119,1381,4846,0.285,475,5403,0.088,622,5403,0.115


## Pitching Feature engineering ideas
- Pitching stats will focus on starting pitchers Per9 innings stat (except for certain features)
    - 'winningPercentage'
    - 'runsScoredPer9'
        - 'ops' - may leave out for now...
    - 'hitsPer9'
    - 'era'
    - 'whip'

In [19]:
pitching_df.head()

,year,playerName,teamAbbrev,winningPercentage,runsScoredPer9,ops,strikeoutsPer9,baseOnBallsPer9,homeRunsPer9,hitsPer9,gidp,avg,era,whip,winPercentage,strikeoutsPer9Inn,walksPer9Inn,hitsPer9Inn
0,2009,Nick Adenhart,LAA,.---,4.50,0.662,7.50,4.50,0.00,10.50,1,0.292,0.00,1.67,.---,7.50,4.50,10.50
1,2009,Mitch Atkins,CHC,.---,0.00,0.286,0.00,0.00,0.00,4.50,1,0.143,0.00,0.50,.---,0.00,0.00,4.50
2,2009,Ernesto Frieri,SD,.---,0.00,0.143,9.00,4.50,0.00,0.00,0,0.000,0.00,0.50,.---,9.00,4.50,0.00
3,2009,Ross Gload,FLA,.---,9.00,0.500,0.00,18.00,0.00,0.00,1,0.000,0.00,2.00,.---,0.00,18.00,0.00
4,2009,Nick Green,BOS,.---,13.50,0.333,0.00,13.50,0.00,0.00,0,0.000,0.00,1.50,.---,0.00,13.50,0.00


In [20]:
pitching_df = pitching_df.reindex(columns=['year', 'playerName', 'teamAbbrev', 'winningPercentage',
       'runsScoredPer9', 'hitsPer9', 'strikeoutsPer9', 'baseOnBallsPer9',
       'homeRunsPer9', 'era', 'whip', 'ops', 'gidp', 'avg'])
# Columns to be left out from the reindexing --- 'winPercentage', 'strikeoutsPer9Inn', 'walksPer9Inn', 'hitsPer9Inn'

In [21]:
# MIA & FLO are different. Standardizing to MIA
pitching_df.replace(to_replace='FLA', value='MIA', inplace=True)

In [22]:
# 31 teams showing up since it seems like there are some NaN values for team names. Will deal with should it become an issue...
len(pitching_df.teamAbbrev.unique())

31

## Game DF Feature ideas
- Turn VisitorRunsScored & HomeRunsScore into a column that indicates if the home team won - DONE.
- Combine VisitorHBP & VisitorBB & VisitorIBB into a single column containing all the stats - DONE.
- Same with HomeHBP & HomeBB & HomeIBB - DONE.
<br/><br/>
- Combine VisitorH / VisitorAB = team_BA - DONE.
- Same with HomeH / HomeAB - DONE. 
<br/><br/>
- Eventually add on all the other engineered features into this dataframe...
<br/><br/>
- Since the game_df has a date column, I'd like to leverage that to look at 7-day-trends, 14-day-trends, etc

In [24]:
# Creating a Target for the model to eventually train on
game_df['Home_team_won?'] = game_df['HomeRunsScore'] > game_df['VisitorRunsScored']

In [25]:
# game_df['Visitor_HBP+BB+IBB'] = game_df['VisitorHBP'] + game_df['VisitorBB'] + game_df['VisitorIBB']
# game_df['Home_HBP+BB+IBB'] = game_df['HomeHBP'] + game_df['HomeBB'] + game_df['HomeIBB']

# Commenting out for now until I figure out if I need to drop these or not...
# game_df.drop(['VisitorHBP', 'VisitorBB', 'VisitorIBB', 'HomeHBP', 'HomeBB', 'HomeIBB'], axis=1, inplace=True)

In [26]:
# Adding a Home & Away BA per game to compare batting trends over time
game_df['Visitor_BA'] = round((game_df['VisitorH'] / game_df['VisitorAB']), 3)
game_df['Home_BA'] = round((game_df['HomeH'] / game_df['HomeAB']), 3)

In [27]:
# Creating a new column in order to add Date/Time to my dataframe
game_df['New_Date'] = pd.to_datetime(game_df['Date'].astype(str), format='%Y%m%d')

In [28]:
# Rearranging columns so new date/time is at the front of the DF
game_df = game_df.reindex(columns=['New_Date','Date', 'VisitingTeam','VisitorStartingPitcherName',
                                 'HomeTeam','HomeStartingPitcherName', 'VisitorRunsScored',
       'HomeRunsScore', 'VisitorAB', 'VisitorH','Visitor_BA', 'VisitorHBP', 'VisitorBB',
       'VisitorIBB', 'VisitorLOB', 'HomeAB', 'HomeH', 'Home_BA', 'HomeHBP', 'HomeBB', 'HomeIBB', 'HomeLOB', 'Home_team_won?'])
game_df.drop('Date', axis='columns', inplace=True)

In [29]:
# Had an extra team due to the Marlins having two different labels. Updating their team name to MIA
game_df.replace(to_replace='FLO', value='MIA', inplace=True)
# Updating game_df so team abbreviations match. Will be leveraged when aggregating teams stats
game_df.replace(to_replace={"NYA":"NYY", "SDN":"SD", "CHN":"CHC", "SLN":"STL", "SFN":"SF", "LAN":"LAD", "TBA":"TB", "KCA":"KC", "CHA":"CWS", "ANA":"LAA", "NYN":"NYM"}, inplace=True)

In [30]:
# game_df has 30 teams. Perfect!
len(game_df.VisitingTeam.unique())

30

## Add Pitching Stats to Game_DF

#### Pitching Feature engineering ideas
- Pitching stats will focus on starting pitchers Per9 innings stat (except for certain features)
    - 'winningPercentage'
    - 'runsScoredPer9'
        - 'ops' - may leave out for now...
    - 'hitsPer9'
    - 'era'
    - 'whip'

In [38]:
pitching_df.head()

,year,playerName,teamAbbrev,winningPercentage,runsScoredPer9,hitsPer9,strikeoutsPer9,baseOnBallsPer9,homeRunsPer9,era,whip,ops,gidp,avg
0,2009,Nick Adenhart,LAA,.---,4.50,10.50,7.50,4.50,0.00,0.00,1.67,0.662,1,0.292
1,2009,Mitch Atkins,CHC,.---,0.00,4.50,0.00,0.00,0.00,0.00,0.50,0.286,1,0.143
2,2009,Ernesto Frieri,SD,.---,0.00,0.00,9.00,4.50,0.00,0.00,0.50,0.143,0,0.000
3,2009,Ross Gload,MIA,.---,9.00,0.00,0.00,18.00,0.00,0.00,2.00,0.500,1,0.000
4,2009,Nick Green,BOS,.---,13.50,0.00,0.00,13.50,0.00,0.00,1.50,0.333,0,0.000


In [42]:
# How to Access the
game_df['New_Date'].dt.year

0        2010
1        2010
2        2010
3        2010
4        2010
         ... 
24292    2019
24293    2019
24294    2019
24295    2019
24296    2019
Name: New_Date, Length: 24297, dtype: int64

In [36]:
game_df['prior_year_Visiting_SP_winningPercentage'] = pitching_df[]


,New_Date,VisitingTeam,VisitorStartingPitcherName,HomeTeam,HomeStartingPitcherName,VisitorRunsScored,HomeRunsScore,VisitorAB,VisitorH,Visitor_BA,...,VisitorIBB,VisitorLOB,HomeAB,HomeH,Home_BA,HomeHBP,HomeBB,HomeIBB,HomeLOB,Home_team_won?
0,2010-04-04,NYY,CC Sabathia,BOS,Josh Beckett,7,9,37,12,0.324,...,0,9,34,12,0.353,0,4,0,6,True
1,2010-04-05,MIN,Scott Baker,LAA,Jered Weaver,3,6,32,7,0.219,...,0,7,33,9,0.273,0,3,0,6,True
2,2010-04-05,CLE,Jake Westbrook,CWS,Mark Buehrle,0,6,30,4,0.133,...,0,4,28,6,0.214,2,5,0,5,True
3,2010-04-05,DET,Justin Verlander,KC,Zack Greinke,8,4,39,12,0.308,...,0,6,35,9,0.257,0,2,0,7,False
4,2010-04-05,SEA,Felix Hernandez,OAK,Ben Sheets,5,3,31,6,0.194,...,1,8,28,5,0.179,0,7,0,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24292,2019-09-29,DET,Spencer Turnbull,CWS,Ross Detwiler,3,5,31,7,0.226,...,0,4,28,4,0.143,3,1,0,3,True
24293,2019-09-29,MIN,Martin Perez,KC,Jorge Lopez,4,5,32,6,0.188,...,0,2,33,9,0.273,0,3,0,7,True
24294,2019-09-29,OAK,Tanner Roark,SEA,Justin Dunn,1,3,30,3,0.100,...,0,6,31,7,0.226,0,5,0,9,True
24295,2019-09-29,NYY,Chad Green,TEX,Lance Lynn,1,6,30,3,0.100,...,0,3,31,10,0.323,1,4,0,7,True
